<a href="https://colab.research.google.com/github/Plogeur/HAI923/blob/main/Notebook_GAN%26AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1> Notebook : Création des modèles GAN, AE et VAE </H1>

Bienvenue dans ce notebook dédié à l'exploration en profondeur des générateurs adverses (GANs), des autoencodeurs (AEs) et des autoencodeurs variationnels (VAEs). Les GANs, conceptualisés par Ian Goodfellow et ses collègues en 2014, ont révolutionné la génération de données en introduisant un cadre d'entraînement basé sur la compétition entre un générateur et un discriminateur. Ils ont permis de créer des données synthétiques d'une qualité inégalée, propulsant ainsi des avancées majeures dans des domaines tels que la génération d'images photoréalistes et la synthèse de vidéos. Les autoencodeurs (AE), quant à eux, ont ouvert la voie à des techniques de compression de données innovantes en apprenant à représenter de manière compacte l'information essentielle d'une donnée tout en minimisant la perte d'information. Ils ont été largement utilisés pour des tâches de débruitage, de reconstruction, et de réduction de dimensionnalité. Les autoencodeurs variationnels (VAEs) ont apporté une dimension probabiliste à l'apprentissage de représentations. Ils ont la capacité unique de générer de nouvelles données en explorant un espace latent continu. Cette approche probabiliste a ouvert de nouvelles perspectives dans la génération de données avec une contrôlabilité fine.

Dans ce notebook avancé, nous plongerons profondément dans ces trois paradigmes, explorant leurs mécanismes internes, leurs architectures avancées, ainsi que leurs applications de pointe. Nous discuterons également des défis et des limites actuelles de ces techniques et des pistes de recherche futures.

*Note à moi-même : putain j'ai pas envie de le faire celui-la, ca va être longue...*

https://github.com/yaxingwang/MineGAN/tree/master/styleGANv2

https://github.com/yaxingwang/Transferring-GANs

https://github.com/mshahbazi72/cGANTransfer

# Installation

In [ ]:
import numpy as np
import os
import seaborn as sns
import random
import shutil
import keras
import pandas as pd
from numpy import mean
from numpy import std
import cv2
from keras import backend as K
import matplotlib.cm as cm
import tensorflow as tf
from keras import metrics
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.saving import load_model
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.layers import Dropout, Dense, GlobalAveragePooling2D, Flatten, Rescaling, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, LeakyReLU
import matplotlib.pyplot as plt
import pathlib
import sys

In [ ]:
# GLOBAL VARIABLE
IMG_SIZE = 128
BATCH_SIZE = 8
CHANEL = 3
N_KFOLDS = 10
STOPPING_PATIENCE = 15
REDUCTION_PATIENCE = 5
EPOCHS = 500
VERBOSE = 1
COLUMNS = 25
SEED = 123
NUMBERCLASS = 3
PATH_DIR = "/content/gdrive/MyDrive/Colab_Notebooks/HAI923/Tiger-Fox-Elephant/"

# SET SEED
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

# CALLBACKS
def callbacks(modelName) :
  EARLY_STOPPING = \
          EarlyStopping(
              monitor='val_loss',
              patience=STOPPING_PATIENCE,
              verbose=VERBOSE,
              mode='auto')

  LR_REDUCTION = \
          ReduceLROnPlateau(
              monitor='val_accuracy',
              patience=REDUCTION_PATIENCE,
              verbose=VERBOSE,
              factor=0.2,
              min_lr=0.000001)

  CHECKPOINT = ModelCheckpoint(f"{modelName}.h5", monitor='val_accuracy', verbose=VERBOSE,
      save_best_only=True, mode='auto', save_freq="epoch")

  CALLBACKS = [EARLY_STOPPING, LR_REDUCTION, CHECKPOINT]
  return CALLBACKS

METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.F1Score(threshold=0.5, average='macro', name='f1_score')
      ]

from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/MyDrive/Colab_Notebooks/HAI923/
path = %pwd

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Colab_Notebooks/HAI923


# Preprocessing

In [ ]:
def create_rgb(my_path, my_classes) :
  X,y=create_X_y(my_path, my_classes)
  print("Les classes : ", my_classes)
  print("Nombres de données : ", X.shape[0])
  print ("Résolution des images : ", X[0].shape)
  X=X.astype('float')
  X=X/255.0
  return X,y

def create_training_data(path_data, list_classes):
  training_data=[]
  for classes in list_classes:
    path=os.path.join(path_data, classes)
    class_num=list_classes.index(classes)
    for img in os.listdir(path):
      try :
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_UNCHANGED)
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, class_num])
      except Exception as e:
        pass
  return training_data

def create_X_y (path_data, list_classes):
      training_data=create_training_data(path_data, list_classes)
      random.shuffle(training_data)
      X=[]
      y=[]
      for features, label in training_data:
        X.append(features)
        y.append(label)
      X=np.array(X).reshape(-1,IMG_SIZE, IMG_SIZE, 3)
      #y=to_categorical(y) #onehot
      y=np.array(y)

      return X,y

# Model

In [ ]:
def create_generator(latent_dim=100) :
  epsilon = 0.0001 # Small float added to variance to avoid dividing by zero in the BatchNorm layers.
  noise_shape = (100,)
  size = int(IMG_SIZE/(32*2))
  model = Sequential()

  # Définition des couches de Convolution/Pooling
  model.add(Dense(size*size*512*2, activation='linear', input_shape=noise_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((size, size, 512*2)))

  model.add(Conv2DTranspose(512*2, kernel_size=[4,4], strides=[2,2], padding="same", kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(256*2, kernel_size=[4,4], strides=[2,2], padding="same", kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(128*2, kernel_size=[4,4], strides=[2,2], padding="same", kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(64*2, kernel_size=[4,4], strides=[2,2], padding="same", kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(64, kernel_size=[4,4], strides=[2,2], padding="same", kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(32, kernel_size=[4,4], strides=[2,2], padding="same", kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
  return model

In [ ]:
def create_discriminator():
  model = Sequential()

  # Définition des couches de Convolution/Pooling
  model.add(Conv2D(2*128, (3,3), padding='same', input_shape=[IMG_SIZE,IMG_SIZE,3]))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(2*128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(MaxPooling2D(pool_size=(3,3)))
  model.add(Dropout(0.2))
  model.add(Conv2D(2*128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(2*128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(MaxPooling2D(pool_size=(3,3)))
  model.add(Dropout(0.3))
  model.add(Conv2D(2*128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(2*128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(MaxPooling2D(pool_size=(3,3)))
  model.add(Dropout(0.3))
  model.add(Conv2D(2*128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(2*128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(MaxPooling2D(pool_size=(3,3)))
  model.add(Dropout(0.3))

  # Flattening : passage de matrices 3D vers un vecteur
  model.add(Flatten())
  model.add(Dense(2*128))
  model.add(LeakyReLU(alpha=0.2))

  # Couche de sortie : classification => binaire : 1 -> vraie image, 0 -> fausse image
  model.add(Dense(2*128))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(1, activation='sigmoid'))

  # compilation du  modèle de classification
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
# Chargement et normalisation des données
def load_data(animal):
  my_path="Data_Project/Tiger-Fox-Elephant/"
  my_classes=[animal]
  X,y=create_X_y(my_path,my_classes)
  # Surtout ne pas oublier de normaliser les données avec :
  X=X.astype('float32')
  #X=X/255.0
  X=(X - 127.5) / 127.5
  return X

# Creation d'un jeu de données de vraies images
# les vraies images sont labélisées avec 1
def generate_real_samples(dataset, nb_images):
  # tirage aléatoire
  ix = randint(0, dataset.shape[0], nb_images)
  #serie = random.sample(range(0, dataset.shape[0]), nb_images)
  # sélection des images
  X = dataset[ix]
  # mettre 1 comme label de classe
  y = ones((nb_images, 1))
  return X, y

# Création d'un faux jeu de données
# elles seront labélisées avec 0
def generate_fake_samples(nb_images):
  X = np.random.rand(IMG_SIZE * IMG_SIZE * 3 * nb_images)
  # reshape en images grises
  X = X.reshape((nb_images, IMG_SIZE, IMG_SIZE, 3))
  # mettre 0 comme label de classe
  y = zeros((nb_images, 1))
  return X, y

In [ ]:
# train the discriminator model
def train_discriminator(model, dataset, epochs=100, batchsize=20):
  # on constitue un jeu de données de batchsize/2 images réelles et images fausses
	half_batch = int(batchsize / 2)
	# boucler sur les epochs
	for i in range(epochs):
		# sélection d'images réelles
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# mettre à jour le discriminateur avec les images réelles
		_, real_acc = model.train_on_batch(X_real, y_real)
		# generation de fausses images
		X_fake, y_fake = generate_fake_samples(half_batch)
		# mise à jour du discriminateur avec de fausses images
		_, fake_acc = model.train_on_batch(X_fake, y_fake)
		# Affichage des résultats pour l'accuracy des vraies et des fausses
		print('>%d accuracy_real=%.0f%% accuracy_fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

In [ ]:
def create_gan(generator_model, discriminator_model):
  # mettre les poids du discriminateur non entrable
  discriminator_model.trainable = False
  # création d'un seul modele qui regroupe generateur et discriminateur
  model = Sequential()
  # ajout du générateur
  model.add(generator_model)
  # ajout du discriminateur
  model.add(discriminator_model)

  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

In [ ]:
def generate_latent_points(latent_dim, nb_images):
  X_input = np.random.randn(latent_dim * nb_images)
  X_input = X_input.reshape(nb_images, latent_dim)
  return X_input

def generate_fake_samples_for_generator(generator_model, latent_dim, nb_images):
  # generation des points
  X_input = generate_latent_points (latent_dim,nb_images)
  # prediction de la sortie du générateur
  X = generator_model.predict(X_input)
  # A ce niveau on considère que les images sont fausses
  # donc on met 0 comme label.
  y = zeros((nb_images, 1))
  return X, y

def plot_and_save_generatedimages(generated_images, epoch, nb_images=10):
	# Affichage des images
	for i in range(nb_images * nb_images):
		pyplot.subplot(nb_images, nb_images, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(generated_images[i, :, :, 0], cmap='gray_r')
	# sauvegarde de l'image
	filename = 'generated_plot_FashionCNN_withGan%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

def evaluate_model(dataset, epoch, generator_model, discriminator_model, latent_dim, nb_images=100, save_model=False):
  # récupération de vraies images pour le discriminateur
  X_real, y_real = generate_real_samples(dataset, nb_images)
  # Evaluation de l'accuracy pour le discriminateur
  _, acc_real = discriminator_model.evaluate(X_real, y_real, verbose=0)

  # génération de fausses images pour le générateur et donc le gan
  X_fake, y_fake = generate_fake_samples_for_generator(generator_model, latent_dim, nb_images)
  # Evaluation du discriminateur avec des fausses images
  _, acc_fake = discriminator_model.evaluate(X_fake, y_fake, verbose=0)
  print ('Accuracy reélle : %.0f%%, fausse : %.0f%%' % (acc_real*100, acc_fake*100))

  if save_model==True:
    # sauvegarde du generateur pour un autre usage
    filename = 'generator_model_GAN%03d.h5' % (epoch + 1)
    generator_model.save(filename)

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=25):
  # Pour déterminer combien il y aura de batchs analysés à chaque epoch
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
	# manually enumerate epochs
  for i in range(n_epochs):
		# enumerate batches over the training set
    print("epoche n°",i)
    for j in range(bat_per_epo):
			  # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, half_batch)
        # update discriminator model weights
        d_loss1, _ = d_model.train_on_batch(X_real, y_real, reset_metrics=False)
        # generate 'fake' examples
        X_fake, y_fake = generate_fake_samples_for_generator(g_model, latent_dim, half_batch)
        # update discriminator model weights
        d_loss2, _ = d_model.train_on_batch(X_fake, y_fake, reset_metrics=False)
        # prepare points in latent space as input for the generator
        X_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        # update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch(X_gan, y_gan, reset_metrics=False)

    #affiche tout les 20 epochs les images générés
    if i % 10 == 0 and i > 0 :
      print('>%d, %d/%d, dis_loss_true=%.3f, dis_loss_false=%.3f gan_loss=%.3f' %
      (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
      evaluate_model (dataset, i, generator_model, discriminator_model, latent_dim)
      latent_dim=100
      COLUMNS = 25
      plt.figure(figsize=(15,15))

      for i in range(COLUMNS):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        noise=tf.random.normal([1,latent_dim])
        plt.imshow((generator_model.predict(noise)[0]+1)/2)
      plt.show()

# TL model

In [ ]:
!git clone https://github.com/mshahbazi72/cGANTransfer.git

In [ ]:
!pip install -r requirements.txt